## Imports

In [232]:
import pandas as pd 
import tabula 
from datetime import date

## Get and parse data from WSEE Kraków

### Get data

In [233]:
df_tab = tabula.read_pdf("https://wsse.krakow.pl/page/wp-content/uploads/2020/09/COVID-19-ogniska-stan-na-28.09.2020-MPWIS.pdf")
df_tab = pd.DataFrame(df_tab[0])

'pages' argument isn't specified.Will extract only from page 1 by default.


### Fix improper string parsing

In [234]:
def fix_str(item):
    if isinstance(item, str):
        return item.replace('\r', ' ').replace('1)', '').replace('3)', '')

def fix_series(series):
    return series.apply(fix_str)


df_tab = df_tab.apply(fix_series, axis=0)

### Change labels and trim table

In [235]:
df_tab.iloc[1,0] = df_tab.iloc[0,0]
df_tab.iloc[1,1] = df_tab.iloc[0,1]

df_tab.columns = df_tab.iloc[1]
df_tab.drop(df_tab.index[1], inplace=True)
df_tab.drop(df_tab.index[0], inplace=True)
df_tab[df_tab['Lp.'] == 'Razem Małopolska'] = df_tab[df_tab['Lp.'] == 'Razem Małopolska'].shift(periods = 1, axis = 1)

In [236]:
df_tab = df_tab[['Powiat', 'Ogółem', 'Ludność', 'Liczba aktywnych przypadków']]

## Prepare data for Kraków

### Leave only numerical values for Kraków

In [237]:
df_Krk = df_tab[df_tab['Powiat'] == 'm. Kraków']
df_Krk.drop(columns = ['Powiat'], inplace = True)

In [238]:
df_Krk['Ogółem'] = [int(str(val).replace(' ','')) for val in df_Krk['Ogółem'].values]
df_Krk['Ludność'] = [int(str(val).replace(' ','')) for val in df_Krk['Ludność'].values]
df_Krk['Liczba aktywnych przypadków'] = [int(str(val).replace(' ','')) for val in df_Krk['Liczba aktywnych przypadków'].values]

### Add current date as index

In [253]:
df_Krk['Data'] = date.today()
df_Krk.set_index(df_Krk['Data'], inplace = True, drop = True)
df_Krk.drop(columns = ['Data'], inplace = True)
df_Krk.columns.name = None

### Perform calculations

In [257]:
df_Krk['Aktywnych na 10tys mieszkańców'] = (df_Krk['Liczba aktywnych przypadków'] / df_Krk['Ludność']) * 10000

In [256]:
df_Krk.to_csv('../Data/data.csv')